author: @SteveEckel

A simple script to test the creation of the radiation reaction approximation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylcp
from pylcp.common import spherical2cart
from scipy.integrate import solve_ivp
import time
plt.style.use('paper')

Define a really, really, simple problem:

In [12]:
laserBeams = pylcp.laserBeams([
    {'kvec': np.array([1., 0., 0.]), 'pol':np.array([0., 1., 0.]), 'delta':0., 'beta':2.0}
    ])

# Then the magnetic field:
magField = lambda R: np.zeros(R.shape)

# %%
# Now define the extremely simple Hamiltonian:
"""Hg = np.array([[0.]])
mugq = np.array([[[0.]], [[0.]], [[0.]]])
He = np.array([[0.]])
mueq = np.array([[[0.]], [[0.]], [[0.]]])
dijq = np.array([[[0.]], [[1.]], [[0.]]])

hamiltonian = pylcp.hamiltonian(Hg, He, mugq, mueq, dijq)
hamiltonian.print_structure()"""

"""H_g, muq_g = pylcp.hamiltonians.singleF(F=1, gF=1, muB=1)
H_e, mue_q = pylcp.hamiltonians.singleF(F=2, gF=1, muB=1)
d_q = pylcp.hamiltonians.dqij_two_bare_hyperfine(1, 2)
hamiltonian = pylcp.hamiltonian()
hamiltonian.add_H_0_block('g', H_g)
hamiltonian.add_H_0_block('e', H_e-0.*np.eye(H_e.shape[0]))
hamiltonian.add_d_q_block('g', 'e', d_q, gamma=1.)"""

atom = pylcp.atom("7Li")
H_g_D2, mu_q_g_D2 = pylcp.hamiltonians.hyperfine_coupled(
    atom.state[0].J, atom.I, atom.state[0].gJ, atom.gI,
    atom.state[0].Ahfs/atom.state[2].gammaHz, Bhfs=0, Chfs=0,
    muB=1)
H_e_D2, mu_q_e_D2 = pylcp.hamiltonians.hyperfine_coupled(
    atom.state[2].J, atom.I, atom.state[2].gJ, atom.gI,
    Ahfs=atom.state[2].Ahfs/atom.state[2].gammaHz,
    Bhfs=atom.state[2].Bhfs/atom.state[2].gammaHz, Chfs=0,
    muB=1)

dijq_D2 = pylcp.hamiltonians.dqij_two_hyperfine_manifolds(
    atom.state[0].J, atom.state[2].J, atom.I)

E_e_D2 = np.unique(np.diagonal(H_e_D2))
E_g_D2 = np.unique(np.diagonal(H_g_D2))

hamiltonian_D2 = pylcp.hamiltonian(H_g_D2, H_e_D2, mu_q_g_D2, mu_q_e_D2, dijq_D2)

Define a function for applying the radiation reaction approximation:

In [13]:
obe = pylcp.obe(laserBeams, magField, hamiltonian, use_sparse_matrices=False)

def build_decay_ev(self):
    """
    This method constructs the decay portion of the OBE using the radiation
    reaction approximation.
    """
    d_q_bare = self.hamiltonian.d_q_bare
    d_q_star = self.hamiltonian.d_q_star

    ev_mat = np.zeros((self.hamiltonian.n**2, self.hamiltonian.n**2),
                       dtype='complex128')

    for key in d_q_bare:
        d_q = d_q_bare[key] + d_q_star[key]
        # The first index we want to capture:
        for ii in range(self.hamiltonian.n):
            # The second index we want to capture:
            for jj in range(self.hamiltonian.n):
                # The first sum index:
                for kk in range(self.hamiltonian.n):
                    # The second sum index:
                    for ll in range(self.hamiltonian.n):
                        for mm, q in enumerate(np.arange(-1., 2., 1)):
                            # first term in the commutator, first part:
                            ev_mat[self._obe__density_index(ii, jj),
                                   self._obe__density_index(ll, jj)] -= \
                            d_q_star[key][mm, ll, kk]*d_q_bare[key][mm, kk, ii]
                            # first term in the commutator, second part:
                            ev_mat[self._obe__density_index(ii, jj),
                                   self._obe__density_index(kk, ll)] += \
                            d_q_star[key][mm, kk, ii]*d_q_bare[key][mm, jj, ll]

                            # second term in the commutator, first part:
                            ev_mat[self._obe__density_index(ii, jj),
                                   self._obe__density_index(ll, kk)] += \
                            d_q_star[key][mm, ll, ii]*d_q_bare[key][mm, jj, kk]
                            # second term in the commutator, second part:
                            ev_mat[self._obe__density_index(ii, jj),
                                   self._obe__density_index(ii, ll)] -= \
                            d_q_star[key][mm, jj, kk]*d_q_bare[key][mm, kk, ll]
    return ev_mat

ev_mat = build_decay_ev(obe)
print(ev_mat)
print(obe.ev_mat['decay'])
print(np.allclose(ev_mat/2, obe.ev_mat['decay']))

[[ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 ...
 [ 0.+0.j  0.+0.j  0.+0.j ... -2.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j -2.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j -2.+0.j]]
[[ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 ...
 [ 0.+0.j  0.+0.j  0.+0.j ... -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j -1.+0.j]]
True
